In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import os

# Ensure output directory exists
output_dir = "fulldata/scraped"
os.makedirs(output_dir, exist_ok=True)

# Years to scrape
years = [1971, 1977, 1980, 1984, 1989, 1991, 1996, 2001, 2006, 2011, 2016]


In [2]:
# Dictionaries to store data dynamically
wikipedia_data = {}
elections_india_data = {}

In [3]:


# Function to scrape data from Wikipedia
def scrape_wikipedia(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        soup = BeautifulSoup(response.text, 'html.parser')

        # Process the first table
        tables_html = soup.find_all('table')
        if tables_html:
            table_str = str(tables_html[0])
            table = pd.read_html(StringIO(table_str))[0]
            return table
        else:
            print(f"No tables found on {url}")
            return pd.DataFrame()
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return pd.DataFrame()

# Function to extract data from Elections India website
def scrape_elections_india(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the first table
        table = soup.find('table')
        rows = table.find_all('tr')

        # Extract headers
        headers = [header.text.strip() for header in rows[0].find_all('th')]
        
        # If headers are not correctly identified, manually fix them
        if not headers or len(headers) < 2:
            print(f"Headers are missing or malformed in {url}.")
            headers = ['AC No.', 'Constituency Name', 'Winner Name', 'Gender', 
                       'Party', 'Votes', 'Runner-up Name', 'Gender', 
                       'Party', 'Votes', 'Margin']

        # Extract row data
        data = []
        for row in rows[1:]:
            cols = [col.text.strip() for col in row.find_all('td')]
            data.append(cols)

        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        return df

    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return pd.DataFrame()

# Function to extract all tables from a URL
def extract_tables(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all tables in the page
        tables_html = soup.find_all('table')
        tables = []

        # Iterate over each table and convert to DataFrame
        for i, table_html in enumerate(tables_html):
            table_str = str(table_html)
            try:
                table = pd.read_html(StringIO(table_str))[0]  
                tables.append(table)
            except Exception as e:
                print(f"Skipping table {i} due to error: {e}")

        return tables
    except Exception as e:
        print(f"Error extracting tables from {url}: {e}")
        return []


def clean_2016_data(df):
    # For Year_2016_India_1 (first case: 2 columns with 'Party' and 'Seats' in the first row)
    if len(df.columns) == 2 and df.iloc[0, 0].lower() == 'party':
        df.columns = ['Party', 'Seats']
        df = df[1:]  # Drop the first row containing duplicate headers
        df.rename(columns={"Party":"Paty Name"},inplace=True)
        df.reset_index(drop=True, inplace=True)
    
    # Check if the last row contains 'Party' and 'Seats' and should be the header
    if 'Party' in df.iloc[-1, 0] and 'Seats' in df.iloc[-1, 1]:
        new_header = df.iloc[-1]  # The last row becomes the header
        df = df[:-1]  # Remove the last row
        df.columns = new_header  # Set the new header
        df.rename(columns={"Party":"Paty Name"},inplace=True)
        df.reset_index(drop=True, inplace=True)
        

    # For Year_2016_India_2 (second case: columns more than 10 but without 'AC No.' in columns)
    if len(df.columns) > 10 and 'AC No.' not in df.columns:
        df.columns = ['A. C. NO.', 'Assembly Constituency Name', 'Winner Candidates Name', 'Gender', 
                      'Party', 'Vote', 'Runner-up Candidates Name', 'Gender.1', 
                      'Party.1', 'vote', 'Margin']
        
        # Add an empty 'Category' column next to 'Assembly Constituency Name'
        df.insert(df.columns.get_loc('Assembly Constituency Name') + 1, 'Category', '')

        # Drop the 'Margin' column
        df.drop(columns=['Margin'], inplace=True)

    return df

# Function to flatten multi-index columns
def flatten_columns(df):
    """Flattens MultiIndex columns in a DataFrame."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(col).strip() for col in df.columns.values]
    return df



In [4]:
# Loop through the years and scrape the data
for year in years:
    print(f"Scraping data for year: {year}")
    
    # Construct URLs dynamically
    url_wikipedia = f"https://en.wikipedia.org/wiki/{year}_Tamil_Nadu_Legislative_Assembly_election"
    url_elections_india = f"https://www.elections.in/tamil-nadu/assembly-constituencies/{year}-election-results.html"

    # Scrape Wikipedia and Elections India data
    wiki_data = scrape_wikipedia(url_wikipedia)
    elections_india_df = scrape_elections_india(url_elections_india)

    # Store the data in dictionaries
    wikipedia_data[year] = wiki_data
    elections_india_data[year] = elections_india_df

    # Print a preview of the data
    print("\nElections India Data:\n", elections_india_df.head())
    print("\nWikipedia Data:\n", wiki_data.head())

    # Extract tables from both links
    wikipedia_tables = extract_tables(url_wikipedia)
    elections_india_tables = extract_tables(url_elections_india)

    # Store the extracted tables in the dictionary
    wikipedia_data[year] = wikipedia_tables
    elections_india_data[year] = elections_india_tables

Scraping data for year: 1971

Elections India Data:
                           Paty Name Seats
0     Communist Party of India(CPI)     8
1    Dravida Munnetra Kazhagam(DMK)   184
2       All India Forward Bloc(FBL)     7
3                  Independent(IND)     8
4  Indian Union Muslim League (MUL)     2

Wikipedia Data:
                                                    0  \
0                                                NaN   
1                       ← 1967 1–7 March 1971 1977 →   
2                                             ← 1967   
3  All 234 seats in the Legislature of Tamil Nadu...   
4                                            Turnout   

                                                   1       2  
0                                                NaN     NaN  
1                       ← 1967 1–7 March 1971 1977 →     NaN  
2                                     1–7 March 1971  1977 →  
3  All 234 seats in the Legislature of Tamil Nadu...     NaN  
4                        

In [5]:
# Apply cleanup for Year 2016 data
for year in years:
    for idx, table in enumerate(elections_india_data[year]):
        if year == 2016:
            elections_india_data[year][idx] = clean_2016_data(table)

C:\Users\harsh\AppData\Local\Temp\ipykernel_10096\4051255523.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"Party":"Paty Name"},inplace=True)


In [6]:
# Writing the data to Excel with flattened columns
with pd.ExcelWriter(os.path.join(output_dir, "elections_wiki_data.xlsx"), engine='xlsxwriter') as writer:
    for year, tables in wikipedia_data.items():
        for idx, table in enumerate(tables):
            try:
                table = flatten_columns(table)
                sheet_name = f"Year_{year}_Wiki_{idx+1}"
                if len(sheet_name) > 31:
                    sheet_name = f"Yr_{year}_Tbl_{idx+1}"
                table.to_excel(writer, sheet_name=sheet_name, index=False)
            except Exception as e:
                print(f"Error writing table for year {year}, index {idx}: {e}")

with pd.ExcelWriter(os.path.join(output_dir, "elections_india_data.xlsx"), engine='xlsxwriter') as writer:
    for year, tables in elections_india_data.items():
        for idx, table in enumerate(tables):
            try:
                table = flatten_columns(table)
                sheet_name = f"Year_{year}_India_{idx+1}"
                if len(sheet_name) > 31:
                    sheet_name = f"Yr_{year}_Tbl_{idx+1}"
                table.to_excel(writer, sheet_name=sheet_name, index=False)
            except Exception as e:
                print(f"Error writing table for year {year}, index {idx}: {e}")

In [7]:
# Checking for MultiIndex columns
for year, tables in wikipedia_data.items():
    for idx, table in enumerate(tables):
        if isinstance(table.columns, pd.MultiIndex):
            print(f"Year {year}, Table {idx+1} has MultiIndex columns.")